<a href="https://colab.research.google.com/github/lgsilva-dev/credito/blob/main/create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criação do Dataset para Concessão e Cálculo de Limite de Crédito.

* Atenção: Dados sintéticos! Devem somente ser utilizados para Cases.

* As regras para criação dos dados podem ser encontradas nesta planilha [Fluxo de Criação XLSX](https://gocn-my.sharepoint.com/:x:/g/personal/lsilva_gocn_com_br/EZ0-itG9yatIqzGs_VG-EVEBep70coN38dOsIha2j35H8A?e=ybWTj6).

* As Regras foram criadas baseadas em artigos academicos e estudos de empresas reais.

* Colunas/Indicadores utilizados também foram escolhidos a partir de necessidades reais de uma empresa ao analisar crédito PJ.

In [ ]:
import pandas as pd
import numpy as np

# --- 0. CONFIGURAÇÃO INICIAL ---
NUM_EMPRESAS = 10000 # Defina o número de empresas que você quer gerar

# Pesos dos indicadores para o cálculo do Score Go On
PESOS = {
    'Margem EBITDA': 0.20,
    'Dívida Líquida/EBITDA': 0.10,
    'Índice de Liquidez': 0.10,
    'Ciclo Financeiro': 0.10,
    'Conversão de EBITDA em FCO': 0.10,
    'Auditoria': 0.10,
    'Tempo de Atuação': 0.05,
    'Pendências Financeiras': 0.05,
    'Garantias': 0.05,
    'Seguros': 0.05,
    'Serasa Score': 0.10,
}

# --- REGRAS DE GERAÇÃO COMBINADA (PORTE vs PERFIL) ---
# Estas matrizes são o "cérebro" da nova lógica

REGRAS_MARGEM_EBITDA = {
    'ME':           {'Bom': (0.05, 0.10), 'Pessimo': (-0.05, 0.029)},
    'EPP':          {'Bom': (0.11, 0.15), 'Pessimo': (-0.03, 0.029)},
    'Médio Porte':  {'Bom': (0.16, 0.25), 'Pessimo': (0.0, 0.029)},
    'Grande Empresa':{'Bom': (0.30, 0.45), 'Pessimo': (0.005, 0.029)}
}

REGRAS_DIVIDA_EBITDA = {
    'ME':           {'Bom': (0.5, 2.0), 'Pessimo': (4.1, 8.0)},
    'EPP':          {'Bom': (1.0, 2.5), 'Pessimo': (4.1, 7.0)},
    'Médio Porte':  {'Bom': (1.5, 3.0), 'Pessimo': (4.1, 6.0)},
    'Grande Empresa':{'Bom': (2.0, 3.5), 'Pessimo': (4.1, 6.0)}
}

REGRAS_LIQUIDEZ = {
    'ME':           {'Bom': (1.5, 3.0), 'Pessimo': (0.5, 0.99)},
    'EPP':          {'Bom': (1.3, 2.5), 'Pessimo': (0.6, 0.99)},
    'Médio Porte':  {'Bom': (1.1, 2.0), 'Pessimo': (0.7, 0.99)},
    'Grande Empresa':{'Bom': (1.0, 1.8), 'Pessimo': (0.6, 0.89)}
}

REGRAS_CICLO_FINANCEIRO = {
    'ME':           {'Bom': (20, 75), 'Pessimo': (100, 200)},
    'EPP':          {'Bom': (15, 60), 'Pessimo': (110, 180)},
    'Médio Porte':  {'Bom': (5, 45),  'Pessimo': (100, 160)},
    'Grande Empresa':{'Bom': (-20, 30),'Pessimo': (90, 150)}
}


# --- INÍCIO DA GERAÇÃO ---
lista_empresas = []
for i in range(NUM_EMPRESAS):
    empresa = {}

    # --- FASE 1: PERFIL DA EMPRESA ---
    empresa['ID_Empresa'] = i + 1

    # Sorteia o perfil para criar uma base com 60% de empresas boas e 40% péssimas.
    perfil_saude = np.random.choice(['Bom', 'Pessimo'], p=[0.60, 0.40])
    empresa['Perfil_Saude_Oculto'] = perfil_saude

    faturamento_log = np.random.normal(loc=15, scale=2.5)
    empresa['Faturamento_Anual'] = np.exp(faturamento_log)

    if empresa['Faturamento_Anual'] <= 360000:
        empresa['Tipo_de_Empresa'] = 'ME'
    elif empresa['Faturamento_Anual'] <= 4800000:
        empresa['Tipo_de_Empresa'] = 'EPP'
    elif empresa['Faturamento_Anual'] <= 300000000:
        empresa['Tipo_de_Empresa'] = 'Médio Porte'
    else:
        empresa['Tipo_de_Empresa'] = 'Grande Empresa'

    porte_empresa = empresa['Tipo_de_Empresa']

    faixas_tempo = {'ME': (1, 5), 'EPP': (3, 10), 'Médio Porte': (8, 25), 'Grande Empresa': (20, 60)}
    min_t, max_t = faixas_tempo[porte_empresa]
    empresa['Tempo_de_Atuacao_Anos'] = int(np.random.uniform(min_t, max_t)) if perfil_saude == 'Bom' else np.random.uniform(0, 2)


    # --- FASE 2: INDICADORES FINANCEIROS BRUTOS (USANDO A LÓGICA COMBINADA) ---
    # Margem EBITDA
    min_m, max_m = REGRAS_MARGEM_EBITDA[porte_empresa][perfil_saude]
    empresa['Margem_EBITDA'] = np.random.uniform(min_m, max_m)
    empresa['EBITDA'] = empresa['Faturamento_Anual'] * empresa['Margem_EBITDA']

    # Depreciação e Amortização
    min_da, max_da = (0.01, 0.05)
    empresa['Depreciacao_Amortizacao'] = empresa['Faturamento_Anual'] * np.random.uniform(min_da, max_da)
    empresa['Lucro_Operacional_EBIT'] = empresa['EBITDA'] - empresa['Depreciacao_Amortizacao']

    # Caixa e Equivalentes
    min_c, max_c = (0.02, 0.15)
    empresa['Caixa_e_Equivalentes'] = empresa['Faturamento_Anual'] * np.random.uniform(min_c, max_c)

    # Dívida Bruta e Líquida
    min_d, max_d = REGRAS_DIVIDA_EBITDA[porte_empresa][perfil_saude]
    multiplo_alavancagem = np.random.uniform(min_d, max_d)
    empresa['Divida_Bruta'] = empresa['EBITDA'] * multiplo_alavancagem
    empresa['Divida_Liquida'] = empresa['Divida_Bruta'] - empresa['Caixa_e_Equivalentes']

    # Ativo e Passivo Circulante
    indice_liquidez_gerado = np.random.uniform(*REGRAS_LIQUIDEZ[porte_empresa][perfil_saude])
    fracao_adicional_ativo = np.random.uniform(0.1, 0.4)
    empresa['Ativo_Circulante'] = empresa['Caixa_e_Equivalentes'] + (empresa['Faturamento_Anual'] * fracao_adicional_ativo)
    empresa['Passivo_Circulante'] = empresa['Ativo_Circulante'] / indice_liquidez_gerado if indice_liquidez_gerado > 0 else empresa['Ativo_Circulante']


    # --- FASE 3: INDICADORES DE PERFORMANCE (KPIs) ---
    empresa['Indice_Divida_Liquida_EBITDA'] = empresa['Divida_Liquida'] / empresa['EBITDA'] if empresa['EBITDA'] != 0 else 999
    empresa['Indice_de_Liquidez_Corrente'] = indice_liquidez_gerado

    faixas_conv = {'Bom': (0.7, 1.1), 'Pessimo': (-0.1, 0.35)}
    empresa['Conversao_EBITDA_em_FCO'] = np.random.uniform(*faixas_conv[perfil_saude])

    min_ciclo, max_ciclo = REGRAS_CICLO_FINANCEIRO[porte_empresa][perfil_saude]
    empresa['Ciclo_Financeiro_Dias'] = int(np.random.uniform(min_ciclo, max_ciclo))


    # --- FASE 4: DADOS QUALITATIVOS E EXTERNOS ---
    if perfil_saude == 'Bom':
        empresa['Pendencias_Financeiras'] = np.random.choice(['Sem pendências', 'Boa Vista'], p=[0.9, 0.1])
        empresa['Auditoria'] = np.random.choice(['Big Four', 'Outra', 'Não Possui'], p=[0.6, 0.2, 0.2]) if porte_empresa != 'ME' else 'Não Possui'
        empresa['Serasa_Score_PJ'] = np.random.randint(650, 1000)
        empresa['Garantias'] = 'Sim'
        empresa['Seguros'] = 'Sim'
    else: # Pessimo
        empresa['Pendencias_Financeiras'] = np.random.choice(['Protesto', 'Ações judiciais', 'Refin'])
        empresa['Auditoria'] = 'Não Possui'
        empresa['Serasa_Score_PJ'] = np.random.randint(100, 299)
        empresa['Garantias'] = 'Não'
        empresa['Seguros'] = 'Não'


        # --- FASE 5: SIMULAÇÃO DO MODELO DE RATING INTERNO ---
    notas = {}

    # Cálculo das notas individuais
    if empresa['Margem_EBITDA'] > 0.12: notas['Margem EBITDA'] = 5
    elif empresa['Margem_EBITDA'] > 0.09: notas['Margem EBITDA'] = 4
    elif empresa['Margem_EBITDA'] > 0.06: notas['Margem EBITDA'] = 3
    elif empresa['Margem_EBITDA'] > 0.03: notas['Margem EBITDA'] = 2
    else: notas['Margem EBITDA'] = 1

    if empresa['Indice_Divida_Liquida_EBITDA'] < 1.0: notas['Dívida Líquida/EBITDA'] = 5
    elif empresa['Indice_Divida_Liquida_EBITDA'] < 2.0: notas['Dívida Líquida/EBITDA'] = 4
    elif empresa['Indice_Divida_Liquida_EBITDA'] < 3.0: notas['Dívida Líquida/EBITDA'] = 3
    elif empresa['Indice_Divida_Liquida_EBITDA'] < 4.0: notas['Dívida Líquida/EBITDA'] = 2
    else: notas['Dívida Líquida/EBITDA'] = 1

    notas['Índice de Liquidez'] = np.digitize(empresa['Indice_de_Liquidez_Corrente'], bins=[0, 1, 1.5, 2, 2.5])
    notas['Ciclo Financeiro'] = 6 - np.digitize(empresa['Ciclo_Financeiro_Dias'], bins=[-100, 0, 30, 60, 90], right=True)
    notas['Conversão de EBITDA em FCO'] = np.digitize(empresa['Conversao_EBITDA_em_FCO'], bins=[0, 0.4, 0.6, 0.8, 0.95])
    notas['Tempo de Atuação'] = np.digitize(empresa['Tempo_de_Atuacao_Anos'], bins=[0, 1, 3, 7, 15])

    nota_pendencia_map = {'Sem pendências': 5, 'Boa Vista': 4}
    notas['Pendências Financeiras'] = nota_pendencia_map.get(empresa['Pendencias_Financeiras'], 1)

    nota_auditoria_map = {'Big Four': 5, 'Outra': 3, 'Não Possui': 1}
    notas['Auditoria'] = nota_auditoria_map.get(empresa['Auditoria'], 1)

    notas['Garantias'] = 5 if empresa['Garantias'] == 'Sim' else 1
    notas['Seguros'] = 5 if empresa['Seguros'] == 'Sim' else 3
    notas['Serasa Score'] = np.digitize(empresa['Serasa_Score_PJ'], bins=[0, 300, 500, 700, 900])

    # === NOVA SEÇÃO: Adicionando as colunas de notas ao dataset ===
    for indicador, nota_valor in notas.items():
        # Cria um nome de coluna limpo, ex: 'Dívida Líquida/EBITDA' -> 'Nota_Dívida_Líquida_EBITDA'
        nome_coluna_nota = f"Nota_{indicador.replace(' ', '_').replace('/', '_')}"
        empresa[nome_coluna_nota] = nota_valor

    # Cálculo do Score Ponderado
    score_go_on = sum(nota_valor * PESOS.get(indicador, 0) for indicador, nota_valor in notas.items())
    empresa['Score_Go_On'] = score_go_on

    # Cálculo do Rating
    if empresa['Score_Go_On'] >= 4.2: empresa['Rating_Go_On'] = '5 (Excelente)'
    elif empresa['Score_Go_On'] >= 3.4: empresa['Rating_Go_On'] = '4 (Bom)'
    elif empresa['Score_Go_On'] >= 2.6: empresa['Rating_Go_On'] = '3 (Regular)'
    elif empresa['Score_Go_On'] >= 1.8: empresa['Rating_Go_On'] = '2 (Ruim)'
    else: empresa['Rating_Go_On'] = '1 (Péssimo)'


    # --- FASE 6: GERAÇÃO DA VARIÁVEL ALVO (TARGET) ---
    if empresa['Score_Go_On'] >= 2.6:
        empresa['Credito_Aprovado'] = 1
        fator_limite = np.interp(empresa['Score_Go_On'], [2.6, 5.0], [0.5, 2.0])
        empresa['Limite_de_Credito'] = abs(empresa['EBITDA'] * fator_limite) if empresa['EBITDA'] > 0 else 0
    else:
        empresa['Credito_Aprovado'] = 0
        empresa['Limite_de_Credito'] = 0

    prob_inadimplencia = 1 / (1 + np.exp((empresa['Score_Go_On'] - 2.8) * 2))
    empresa['Inadimplente_12M'] = np.random.choice([1, 0], p=[prob_inadimplencia, 1 - prob_inadimplencia])

    lista_empresas.append(empresa)

# --- FINALIZAÇÃO ---
df_final = pd.DataFrame(lista_empresas)

# Lista de colunas de notas para organizar a saída
colunas_notas = [f"Nota_{ind.replace(' ', '_').replace('/', '_')}" for ind in PESOS.keys()]

# Organizar colunas para melhor visualização
ordem_colunas = [
    'ID_Empresa', 'Tipo_de_Empresa', 'Faturamento_Anual', 'Tempo_de_Atuacao_Anos',
    'EBITDA', 'Margem_EBITDA', 'Indice_Divida_Liquida_EBITDA', 'Indice_de_Liquidez_Corrente',
    'Ciclo_Financeiro_Dias', 'Conversao_EBITDA_em_FCO', 'Serasa_Score_PJ',
    'Pendencias_Financeiras', 'Auditoria', 'Garantias', 'Seguros',
] + colunas_notas + [ # Adiciona as colunas de notas aqui
    'Score_Go_On', 'Rating_Go_On', 'Credito_Aprovado', 'Limite_de_Credito', 'Inadimplente_12M', 'Perfil_Saude_Oculto'
]

# Pega apenas as colunas que realmente existem para evitar erros
colunas_finais_existentes = [col for col in ordem_colunas if col in df_final.columns]
df_final = df_final[colunas_finais_existentes]

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df_final

,ID_Empresa,Tipo_de_Empresa,Faturamento_Anual,Tempo_de_Atuacao_Anos,EBITDA,Margem_EBITDA,Indice_Divida_Liquida_EBITDA,Indice_de_Liquidez_Corrente,Ciclo_Financeiro_Dias,Conversao_EBITDA_em_FCO,Serasa_Score_PJ,Pendencias_Financeiras,Auditoria,Garantias,Seguros,Nota_Margem_EBITDA,Nota_Dívida_Líquida_EBITDA,Nota_Índice_de_Liquidez,Nota_Ciclo_Financeiro,Nota_Conversão_de_EBITDA_em_FCO,Nota_Auditoria,Nota_Tempo_de_Atuação,Nota_Pendências_Financeiras,Nota_Garantias,Nota_Seguros,Nota_Serasa_Score,Score_Go_On,Rating_Go_On,Credito_Aprovado,Limite_de_Credito,Inadimplente_12M,Perfil_Saude_Oculto
0,1,EPP,5.698315e+05,8.000000,6.643560e+04,0.116588,0.684094,1.530036,43,0.872436,694,Boa Vista,Outra,Sim,Sim,4,5,3,3,4,3,4,4,5,5,3,3.85,4 (Bom),1,8.512061e+04,1,Bom
1,2,EPP,2.964366e+06,5.000000,3.699616e+05,0.124803,0.967256,1.611599,49,0.729814,740,Sem pendências,Big Four,Sim,Sim,5,5,3,3,3,5,3,5,5,5,4,4.25,5 (Excelente),1,5.665037e+05,0,Bom
2,3,ME,1.795568e+04,1.000000,1.223395e+03,0.068134,-0.282689,2.682491,27,0.990045,984,Boa Vista,Não Possui,Sim,Sim,3,5,5,4,5,1,2,4,5,5,5,3.85,4 (Bom),1,1.567475e+03,0,Bom
3,4,Médio Porte,3.604391e+07,1.218148,3.169400e+05,0.008793,-4.136692,0.904970,144,0.238131,254,Protesto,Não Possui,Não,Não,1,5,1,1,1,1,2,1,1,3,1,1.55,1 (Péssimo),0,0.000000e+00,1,Pessimo
4,5,ME,2.020546e+04,1.000000,1.642297e+03,0.081280,0.819650,2.464692,74,0.740844,974,Boa Vista,Não Possui,Sim,Sim,3,5,4,2,3,1,2,4,5,5,5,3.35,3 (Regular),1,1.590975e+03,0,Bom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,Médio Porte,9.725171e+07,1.082831,1.731860e+06,0.017808,0.592488,0.944418,123,-0.009296,167,Refin,Não Possui,Não,Não,1,5,1,1,0,1,2,1,1,3,1,1.45,1 (Péssimo),0,0.000000e+00,1,Pessimo
9996,9997,Médio Porte,1.767790e+07,23.000000,3.268555e+06,0.184895,2.055539,1.240128,42,0.717771,878,Sem pendências,Big Four,Sim,Sim,5,3,2,3,3,5,5,5,5,5,4,4.05,4 (Bom),1,4.596406e+06,0,Bom
9997,9998,EPP,2.017384e+06,3.000000,2.005519e+05,0.099412,0.363798,1.572529,18,0.943036,857,Sem pendências,Outra,Sim,Sim,4,5,3,4,4,3,3,5,5,5,4,4.05,4 (Bom),1,2.820261e+05,0,Bom
9998,9999,EPP,4.090242e+05,7.000000,5.082540e+04,0.124260,0.955328,1.467359,44,0.931388,933,Sem pendências,Outra,Sim,Sim,5,5,2,3,4,3,4,5,5,5,5,4.15,4 (Bom),1,7.464981e+04,0,Bom


In [ ]:
df_final['Credito_Aprovado'].value_counts()

,count
Credito_Aprovado,
1,5978
0,4022


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID_Empresa                       10000 non-null  int64  
 1   Tipo_de_Empresa                  10000 non-null  object 
 2   Faturamento_Anual                10000 non-null  float64
 3   Tempo_de_Atuacao_Anos            10000 non-null  float64
 4   EBITDA                           10000 non-null  float64
 5   Margem_EBITDA                    10000 non-null  float64
 6   Indice_Divida_Liquida_EBITDA     10000 non-null  float64
 7   Indice_de_Liquidez_Corrente      10000 non-null  float64
 8   Ciclo_Financeiro_Dias            10000 non-null  int64  
 9   Conversao_EBITDA_em_FCO          10000 non-null  float64
 10  Serasa_Score_PJ                  10000 non-null  int64  
 11  Pendencias_Financeiras           10000 non-null  object 
 12  Auditoria          

In [ ]:

# Validações
taxa_aprovacao = df_final['Credito_Aprovado'].value_counts(normalize=True).get(1, 0)
print(f"Taxa de Aprovação no dataset: {taxa_aprovacao:.2%}")

dist_perfis = df_final['Perfil_Saude_Oculto'].value_counts(normalize=True)
print("\nDistribuição dos Perfis de Saúde:")
print(dist_perfis)

dist_rating = df_final['Rating_Go_On'].value_counts(normalize=True).sort_index()
print("\nDistribuição dos Ratings Finais:")
print(dist_rating)

Taxa de Aprovação no dataset: 59.78%

Distribuição dos Perfis de Saúde:
Perfil_Saude_Oculto
Bom        0.5978
Pessimo    0.4022
Name: proportion, dtype: float64

Distribuição dos Ratings Finais:
Rating_Go_On
1 (Péssimo)      0.4022
3 (Regular)      0.0090
4 (Bom)          0.3425
5 (Excelente)    0.2463
Name: proportion, dtype: float64
